In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Load data from PostgreSQL
query = 'SELECT * FROM public.overall_cleaned_base_and_pr_ef_policyef;'
data = pd.read_sql(query, con=engine)

In [2]:
# Convert Policy End Date to datetime
data['policy end date'] = pd.to_datetime(data['policy end date'], errors='coerce')

# Separate Open Customers (January to March 2025)
open_customers = data[
    (data['Policy Status'] == 'Open') &
    (data['policy end date'].dt.year == 2025) &
    (data['policy end date'].dt.month.isin([1, 2, 3, 4, 5, 6]))
].copy()


selected_columns = ['policy no', 'renewal type', 'product name', 'product name 2',  'biztype', 'policy end date', 'policy start date', 
 'age', 'manufacturer/make', 'model', 'variant', 'vehicle segment', 'fuel type', 'rto location', 'vehicle idv', 'ncb amount', 'Cleaned Reg no', 
 'before gst add-on gwp', 'total od premium', 'total tp premium', 'gst', 'total premium payable', 
 'ncb % previous year', 'applicable discount with ncb', 'Cleaned Branch Name 2', 'Cleaned State2', 'Cleaned Zone 2', 'tie up',
 'Number of claims', 'approved', 'denied', 'corrected_name', 'customerid', 'Policy Status', 'Policy Tenure', 'Customer Tenure', 'New Customers', 'Claim Happaned/Not', 
 'Renewal Rate Status', 'withdrawn', 'chassis_engine_key', 'policy_wise_purchase']

data = data[selected_columns]

# Filter the main dataset for customers whose Policy End Date is <= December 2024
data = data[data['Policy Status'].isin(['Renewed', 'Not Renewed'])]

# Map Policy Status to binary
data['Policy Status'] = data['Policy Status'].apply(lambda x: 1 if x == 'Not Renewed' else 0)

# Handle missing values
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].fillna('missing')
    else:
        data[column] = data[column].fillna(0)

# Extract year, month, and day from date columns
date_columns = ['policy start date', 'policy end date']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Extract date features
for col in date_columns:
    data[f'{col}_YEAR'] = data[col].dt.year
    data[f'{col}_MONTH'] = data[col].dt.month
    data[f'{col}_DAY'] = data[col].dt.day

# Drop original date columns
data.drop(columns=date_columns, inplace=True)

# Separate features and target variable for training
features = [col for col in data.columns if col != 'Policy Status']
X = data[features]
y = data['Policy Status']

from imblearn.over_sampling import RandomOverSampler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, log_loss, roc_curve
import matplotlib.pyplot as plt

# # Initialize RandomOverSampler
# ros = RandomOverSampler(random_state=42)

# # Apply Random Oversampling to the training data
# X, y = ros.fit_resample(X, y)

# Apply the same transformations to open_customers
for col in date_columns:
    open_customers[col] = pd.to_datetime(open_customers[col], errors='coerce')

for col in date_columns:
    open_customers[f'{col}_YEAR'] = open_customers[col].dt.year
    open_customers[f'{col}_MONTH'] = open_customers[col].dt.month
    open_customers[f'{col}_DAY'] = open_customers[col].dt.day

open_customers.drop(columns=date_columns, inplace=True)

for column in open_customers.columns:
    if open_customers[column].dtype == 'object':
        open_customers[column] = open_customers[column].fillna('missing')
    else:
        open_customers[column] = open_customers[column].fillna(0)

open_customers_without_encoded = open_customers.copy()

C:\Users\Admin\AppData\Local\Temp\ipykernel_7360\2106531804.py:79: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  open_customers[column] = open_customers[column].fillna(0)


In [3]:
# Apply label encoding to categorical features
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':
        label_encoder = LabelEncoder()
        X[column] = label_encoder.fit_transform(X[column].astype(str))
        label_encoders[column] = label_encoder
        
        mapping_dict = {label: i for i, label in enumerate(label_encoder.classes_)}
        next_unique_value = max(mapping_dict.values()) + 1  

        def encode_test_value(value):
            return mapping_dict.get(value, next_unique_value)

        open_customers[column] = open_customers[column].apply(encode_test_value)

C:\Users\Admin\AppData\Local\Temp\ipykernel_7360\3024449356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column].astype(str))
C:\Users\Admin\AppData\Local\Temp\ipykernel_7360\3024449356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column].astype(str))
C:\Users\Admin\AppData\Local\Temp\ipykernel_7360\3024449356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [4]:
# Import necessary libraries
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt

# Define the models
model_1 = xgb.XGBClassifier(
    max_depth=6,                  
    learning_rate=0.1,            
    n_estimators=100,            
    scale_pos_weight=len(y[y == 0]) / len(y[y == 1]), 
    random_state=42
)
model_2 = xgb.XGBClassifier(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

model_3 = xgb.XGBClassifier(
    max_depth=5,                  
    learning_rate=0.05,            
    n_estimators=200,              
    subsample=0.8,                 
    colsample_bytree=0.8,         
    scale_pos_weight=len(y[y == 0]) / len(y[y == 1]),  
    gamma=0.1,                    
    random_state=42
)

model_4 = xgb.XGBClassifier(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=1.0,  # L2 regularization
    random_state=42
)

# Create the Voting Classifier
model = VotingClassifier(
    estimators=[
        ('xgb1', model_1),
        ('xgb2', model_2),
        ('xgb3', model_3),
        ('xgb4', model_4)
    ],
    voting='soft',  # Soft voting for probability averaging
    weights=[0.2497, 0.2497, 0.2472, 0.2534]  # Weight distribution

)

model.fit(X, y)

# Predict using encoded data
X_open_customers = open_customers[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

Predicted Renewed: 264365
Predicted Not Renewed: 94508
Train Accuracy: 0.730175500693939
Train Log Loss: 0.5202055007478678
Train ROC AUC: 0.8020341149319554
Train Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.67      0.65    555376
           1       0.80      0.77      0.78    947638

    accuracy                           0.73   1503014
   macro avg       0.71      0.72      0.71   1503014
weighted avg       0.73      0.73      0.73   1503014

Class 0 Train Accuracy: 0.6655292990694592
Class 1 Train Accuracy: 0.7680622769454158


In [5]:
# Save predictions
open_customers_without_encoded.to_csv('JFM_predictions_xgb 1.csv', index=False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
# Import necessary libraries
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt

model = xgb.XGBClassifier(
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=1.0,  # L2 regularization
    random_state=42
)

model.fit(X, y)

# Predict using encoded data
X_open_customers = open_customers[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
# Import necessary libraries
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt

model = xgb.XGBClassifier(
    max_depth=5,                  
    learning_rate=0.05,            
    n_estimators=200,              
    subsample=0.8,                 
    colsample_bytree=0.8,         
    scale_pos_weight=len(y[y == 0]) / len(y[y == 1]),  
    gamma=0.1,                    
    random_state=42
)

model.fit(X, y)

# Predict using encoded data
X_open_customers = open_customers[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Initialize and train the Decision Tree classifier
model = DecisionTreeClassifier(random_state=42, max_depth=10)

model.fit(X, y)

# Predict using encoded data
X_open_customers = open_customers[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, max_depth=10)

model.fit(X, y)

# Predict using encoded data
X_open_customers = open_customers[features]
y_open_pred = model.predict(X_open_customers)
y_open_pred_proba = model.predict_proba(X_open_customers)[:, 1]  

# Store prediction results in unencoded data
open_customers_without_encoded['Predicted Status'] = np.where(y_open_pred == 1, 'Not Renewed', 'Renewed')
open_customers_without_encoded['Churn Probability'] = y_open_pred_proba

print(f"Predicted Renewed: {(y_open_pred == 0).sum()}")
print(f"Predicted Not Renewed: {(y_open_pred == 1).sum()}")

# Evaluate model on training data
y_pred = model.predict(X)
y_pred_proba = model.predict_proba(X)[:, 1]

#Evaluate the model on training data
train_accuracy = accuracy_score(y, y_pred)
train_log_loss = log_loss(y, y_pred_proba)
train_roc_auc = roc_auc_score(y, y_pred_proba)
train_report = classification_report(y, y_pred)

conf_matrix_train = confusion_matrix(y, y_pred)
class_0_accuracy_train = conf_matrix_train[0, 0] / conf_matrix_train[0].sum()
class_1_accuracy_train = conf_matrix_train[1, 1] / conf_matrix_train[1].sum()

#Print the metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Train Log Loss: {train_log_loss}")
print(f"Train ROC AUC: {train_roc_auc}")
print(f"Train Classification Report:\n{train_report}")
print(f"Class 0 Train Accuracy: {class_0_accuracy_train}")
print(f"Class 1 Train Accuracy: {class_1_accuracy_train}")